In [1]:
!pip install  cryptography

In [2]:
nxos={
        "host": "sandbox-nxos-1.cisco.com",
        "port": 22,
        "username": "admin",
        "password": "Admin_1234!",
        "device_type": "cisco_nxos"
    }
nxos

{'host': 'sandbox-nxos-1.cisco.com',
 'port': 22,
 'username': 'admin',
 'password': 'Admin_1234!',
 'device_type': 'cisco_nxos'}

In [3]:
from cryptography.fernet import Fernet
import os
import base64
import hashlib
from getpass import getpass
from netmiko import ConnectHandler

In [4]:
def generate_key(password: str) -> bytes:
    # Generate a key based on the password
    key = hashlib.sha256(password.encode()).digest()
    return base64.urlsafe_b64encode(key)

In [5]:
# User getpass for password
password = getpass()


 ········


In [6]:

# Generate key from password
key = generate_key(password)
key

b'mJLFM50TqUzAM4TmCXmPTkaI2s-Vu4lv5gkxJV8FTiA='

In [7]:
cipher = Fernet(key)
cipher

In [8]:
# Store credentials in a dictionary
credentials = {
    "device1": {
        "ip": "sandbox-nxos-1.cisco.com",
        "username": "admin",
        "password": getpass()
    }
}
credentials

 ········


{'device1': {'ip': 'sandbox-nxos-1.cisco.com',
  'username': 'admin',
  'password': 'Admin_1234!'}}

In [9]:
for x,y in credentials.items():
    print(x)

device1


In [10]:
for x,y in credentials.items():
    print(y)

{'ip': 'sandbox-nxos-1.cisco.com', 'username': 'admin', 'password': 'Admin_1234!'}


In [11]:
for x,y in credentials.items():
    print(y['ip'])

sandbox-nxos-1.cisco.com


In [12]:
for x,y in credentials.items():
    print(y['username'])

admin


In [13]:
for x,y in credentials.items():
    print(y['password'])

Admin_1234!


In [14]:
# Prepare encrypted credentials
encrypted_credentials = []
for device, info in credentials.items():
    encrypted_password = cipher.encrypt(info['password'].encode()).decode()
    encrypted_credentials.append(f"{device}|{info['ip']}|{info['username']}|{encrypted_password}")
encrypted_credentials

['device1|sandbox-nxos-1.cisco.com|admin|gAAAAABn18wluiS0fjhKRfEa2viz3iQjEDJC3AJorF72S02ShCJuz_INpIdxcj2j8TlxrIcOjaf_1OuxsPEdB6CSLfEGYGymSA==']

In [15]:

# Save encrypted credentials to a text file
with open('credentials.txt', 'w') as file:
    for line in encrypted_credentials:
        file.write(line + '\n')

In [16]:
# Encrypt the entire credentials file with password protection
with open('credentials.txt', 'rb') as file:
    original_data = file.read()
original_data

b'device1|sandbox-nxos-1.cisco.com|admin|gAAAAABn18wluiS0fjhKRfEa2viz3iQjEDJC3AJorF72S02ShCJuz_INpIdxcj2j8TlxrIcOjaf_1OuxsPEdB6CSLfEGYGymSA==\r\n'

In [17]:
encrypted_data = cipher.encrypt(original_data)
encrypted_data

b'gAAAAABn18xFBh7gv6Y9wgXliTIZOZYfs33HAFEpshx9je6SkA3_9AN6lK7d2xWojVaaR3IWfCqSd0vLFbtNrQq5imwd8O-f5UnqZ1rXe6yln6WFxVwx1IEe0pievOWEaJeOYxBFEC_dTRIQ6Js5BcHZrboxjKF0LNseLUPh74KWzs1xEmJQZBuRESSV-4odaJiTxRrftOgUEVqd_qnzuhIbdZdOw-3GEHINppeLYPgVId30jHiUSNx_zBxSUmhfd4kfjqbj6fVA'

In [18]:
# Write the encrypted data to a new file
with open('credentials_encrypted.txt', 'wb') as file:
    file.write(encrypted_data)

In [19]:
# Optionally remove the original credentials file
os.remove('credentials.txt')

print("Credentials have been encrypted and stored in 'credentials_encrypted.txt'.")


Credentials have been encrypted and stored in 'credentials_encrypted.txt'.


In [20]:
!pip install pyzipper

In [21]:
import pyzipper

zip_file = 'credentials_encrypted.zip'
password = 'Admin_1234!'

with pyzipper.AESZipFile(zip_file) as zf:
    zf.setpassword(bytes(password, 'utf-8'))
    zf.extractall()  # Extract all contents


In [22]:
# User input for password
password = getpass()

# Generate key from password
key = generate_key(password)
cipher = Fernet(key)

# Load encrypted credentials from the text file
with open('credentials_encrypted.txt', 'rb') as file:
    encrypted_data = file.read()
print(encrypted_data)


 ········


b'gAAAAABn18xFBh7gv6Y9wgXliTIZOZYfs33HAFEpshx9je6SkA3_9AN6lK7d2xWojVaaR3IWfCqSd0vLFbtNrQq5imwd8O-f5UnqZ1rXe6yln6WFxVwx1IEe0pievOWEaJeOYxBFEC_dTRIQ6Js5BcHZrboxjKF0LNseLUPh74KWzs1xEmJQZBuRESSV-4odaJiTxRrftOgUEVqd_qnzuhIbdZdOw-3GEHINppeLYPgVId30jHiUSNx_zBxSUmhfd4kfjqbj6fVA'


In [23]:
# Decrypt the credentials data
try:
    original_data = cipher.decrypt(encrypted_data).decode()
except Exception as e:
    print("Failed to decrypt the file. Check the password.")
    exit()
print(original_data)

device1|sandbox-nxos-1.cisco.com|admin|gAAAAABn18wluiS0fjhKRfEa2viz3iQjEDJC3AJorF72S02ShCJuz_INpIdxcj2j8TlxrIcOjaf_1OuxsPEdB6CSLfEGYGymSA==



In [24]:


# Split lines and extract device info
lines = original_data.splitlines()
for line in lines:
    device_info = line.strip().split('|')
    device_name, ip, username, encrypted_password = device_info
    
    # Decrypt the password
    decrypted_password = cipher.decrypt(encrypted_password.encode()).decode()

    # Create a device connection
    net_device = {
        'device_type': 'cisco_nxos',  # Change this according to your device type
        'host': ip,
        'username': username,
        'password': decrypted_password,
    }
print(net_device)

{'device_type': 'cisco_nxos', 'host': 'sandbox-nxos-1.cisco.com', 'username': 'admin', 'password': 'Admin_1234!'}
